In [1]:
import tensorflow as tf
from TPM_from_VCS.data import toy_data_generator
from TPM_from_VCS.models.etp import etp
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
tf.enable_eager_execution()

def get_diagram(cost_history, steps):
    plt.plot(np.arange(steps), cost_history)
    plt.xlabel('Epochs')
    plt.ylabel('MSE')
    plt.title('Training Error')
    plt.show()

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

##########################
### WRAPPER FUNCTIONS
##########################

def fully_connected(input_tensor, output_nodes,
                    activation=None, seed=None,
                    name='fully_connected'):

    with tf.name_scope(name):
        input_nodes = input_tensor.get_shape().as_list()[1]
        weights = tf.Variable(tf.truncated_normal(shape=(input_nodes,
                                                         output_nodes),
                                                  mean=0.0,
                                                  stddev=0.1,
                                                  dtype=tf.float32,
                                                  seed=seed),
                              name='weights')
        biases = tf.Variable(tf.zeros(shape=[output_nodes]), name='biases')

        act = tf.matmul(input_tensor, weights) + biases
        if activation is not None:
            act = activation(act)
        return act


##########################
### DATASET
##########################

mnist = input_data.read_data_sets("./", validation_size=0)


##########################
### SETTINGS
##########################


# Hyperparameters
learning_rate = 0.01
training_epochs = 600
batch_size = 128

# Architecture
hidden_size = 2
input_size = 5
# image_width = 28
MAX_VC_DIM = 10
MAX_NODES = 1000

# Other
print_interval = 20
random_seed = 123


##########################
### GRAPH DEFINITION
##########################

g = tf.Graph()
with g.as_default():
    
    tf.set_random_seed(random_seed)
    
#     cost_history = []

    # Input data
    input_layer = tf.placeholder(tf.float32, [None, input_size],
                                 name='input')

    ###########
    # Encoder
    ###########
    
    inp = tf.stack(input_layer, axis=0)
    
    hidden_layer_1 = fully_connected(input_layer, hidden_size * 30, 
                                   activation=None, 
                                   name='hidden0')
    
    hidden_layer0 = fully_connected(hidden_layer_1, hidden_size * 15, 
                                   activation=None, 
                                   name='hidden0')
    
    hidden_layer1 = fully_connected(hidden_layer0, hidden_size * 5, 
                                   activation=None, 
                                   name='hidden1')
    
    hidden_layer2 = fully_connected(hidden_layer1, hidden_size, 
                                   activation=None, 
                                   name='hidden2')
    
#     temp_out = tf.stack(hidden_layer2, name='temp_out', axis=0)
#     temp = tf.identity(temp_out, name='reduced_dim')
    
    
    
    etp_loss = etp.get_best_etp(hidden_layer2, input_layer)
#     loss = tf.identity(etp_loss, 'etp')
    
    
    hidden_layer3 = fully_connected(hidden_layer2, hidden_size * 5, 
                                   activation=None, 
                                   name='hidden3')
    
    hidden_layer4 = fully_connected(hidden_layer3, hidden_size * 15, 
                                   activation=None, 
                                   name='hidden4')
    
    hidden_layer5 = fully_connected(hidden_layer4, hidden_size * 30, 
                                   activation=None, 
                                   name='hidden4')
    
    
    
    ###########
    # Decoder
    ###########
    
    logits = fully_connected(hidden_layer5, input_size, 
                             activation=None, name='logits')
    # note MNIST pixels are normalized to 0-1 range
#     out_layer = tf.nn.sigmoid(logits, name='decoding') 
    
    # set the output layer to be the same as logits. TODO: try relu activation also. Change the cost funcion definition only
    out_layer = tf.nn.relu(logits, name='decoding') 
    
    ##################
    # Loss & Optimizer
    ##################
    
    cost = tf.reduce_mean(tf.losses.mean_squared_error(
        labels=input_layer, predictions=logits), name='cost')
    
#     cost_history.append(cost)
    
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(cost, name='train')

    # Saver to save session for reuse
    saver = tf.train.Saver()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
<class 'NoneType'>
None


TypeError: 'NoneType' object cannot be interpreted as an integer

In [ ]:
batch_x = toy_data_generator.create_dataset(1)

batch_x[0][0]

len(batch_x)


##########################
### TRAINING & EVALUATION
##########################
    
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    cost_history = []
    steps = 0
    
    np.random.seed(random_seed) # random seed for mnist iterator
    for epoch in range(training_epochs):
        avg_cost = 0.
#         total_batch = mnist.train.num_examples // batch_size
        total_batch = 500
        batch_x = toy_data_generator.create_dataset(1)
        
        for i in range(total_batch):
            
            
            for z in range(len(batch_x)):
                _, c = sess.run(['train', 'cost:0'], 
                                feed_dict={'input:0': np.reshape(batch_x[z][0], (-1, 5))})
                avg_cost += c
                
            cost_history.append(c)
            steps+=1
                
            
            if not i % print_interval:
                print("Minibatch: %03d | Cost:    %.3f" % (i + 1, c))
                
            if not steps % 5000:
                get_diagram(cost_history, steps)
                
    
        print("Epoch:     %03d | AvgCost: %.3f" % (epoch + 1, avg_cost / (i + 1)))
    
    saver.save(sess, save_path='./autoencoder.ckpt')

In [ ]:
get_diagram(cost_history, steps)

In [ ]:
a = toy_data_generator.create_dataset(1)
np.reshape(a[0][0], (-1, 5))

In [ ]:
# sess.run(['hidden2'])
ans = []
# saver.restore(sess, save_path='./autoencoder.ckpt')
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
#     saver.restore(sess, save_path='./autoencoder.ckpt')
    for i in range(len(a)):
        
        etp_loss = sess.run(['etp'], feed_dict={'input:0': np.reshape(a[i][0], (-1, 5))})
        
        print('{}'.format(etp_loss))
    

